# Lesson 2: Prompting with o1

## Principles

### 1. Simple and direct

### 2. No explicit CoT required

The first principles we start with are **simple and direct** prompting and avoiding providing **explicit guidance or CoT**. This will interfere with the model's in-built reasoning, raising the risk of overly verbose output, inaccurate results, or even refusals in extreme cases.

In [1]:
import warnings
warnings.filterwarnings("ignore")

from helper import get_openai_api_key
openai_api_key = get_openai_api_key()


In [3]:
from openai import OpenAI
client = OpenAI()
GPT_MODEL = "gpt-4o-mini"
O1_MODEL = "o1-mini"

In [4]:
bad_prompt = ("Generate a function that outputs the SMILES IDs for all the molecules involved in insulin."
              "Think through this step by step, and don't skip any steps:"
              "- Identify all the molecules involve in insulin"
              "- Make the function"
              "- Loop through each molecule, outputting each into the function and returning a SMILES ID"
              "Molecules: ")
response = client.chat.completions.create(model=O1_MODEL,messages=[{"role":"user","content": bad_prompt}])

In [6]:
from IPython.display import display, HTML, Markdown

display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"><h2>🔽 &nbsp; Markdown Output – Beginning</h2></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"><h2>🔼 &nbsp; Markdown Output – End</h2></div>'))


Sure! Let's tackle this step by step to create a function that outputs the SMILES (Simplified Molecular Input Line Entry System) IDs for all the molecules involved in insulin.

### Step 1: Identify All the Molecules Involved in Insulin

**Insulin Structure Overview:**
- **Insulin** is a peptide hormone composed of two polypeptide chains:
  - **A Chain:** 21 amino acids
  - **B Chain:** 30 amino acids
- These two chains are linked together by disulfide bonds.
  
**Molecules Involved:**
1. **Amino Acids:** The building blocks of insulin. Each amino acid has a specific side chain (R-group) that distinguishes it from others.
2. **Disulfide Bonds:** Covalent bonds between the sulfur atoms of cysteine residues, stabilizing the insulin structure.

For the purpose of this function, we'll focus on the amino acids that make up insulin. Representing large proteins like insulin in SMILES is impractical due to their size, so we'll map each amino acid to its respective SMILES string.

### Step 2: Create a Mapping of Amino Acids to SMILES

Each amino acid has a unique SMILES representation. We'll create a dictionary that maps the three-letter or one-letter amino acid codes to their corresponding SMILES strings.

### Step 3: Develop the Function

We'll write a Python function that:
1. Contains a list of the amino acids present in insulin.
2. Uses a dictionary to map each amino acid to its SMILES string.
3. Loops through the list of amino acids, retrieves their SMILES, and outputs them.

### Implementation

Here's how you can implement this in Python:

```python
def get_insulin_smiles():
    """
    Returns the SMILES strings for all amino acids involved in insulin.
    """

    # Mapping of amino acids to their SMILES strings
    amino_acid_smiles = {
        'Ala': 'CC(C)C(=O)O',        # Alanine
        'Arg': 'CNC(CC@N)(=O)O',     # Arginine
        'Asn': 'CC(C(=O)O)N',        # Asparagine
        'Asp': 'CC(C(=O)O)C(=O)O',   # Aspartic Acid
        'Cys': 'C(C(S)C(=O)O)',      # Cysteine
        'Gln': 'CC(C(=O)O)N',        # Glutamine
        'Glu': 'CC(C(=O)O)C(=O)O',   # Glutamic Acid
        'Gly': 'NCC(=O)O',            # Glycine
        'His': 'C1=CN=CN1CC(C(=O)O)N',# Histidine
        'Ile': 'CC(C)CC(C(=O)O)N',    # Isoleucine
        'Leu': 'CC(C)CC(C(=O)O)N',    # Leucine
        'Lys': 'CC(C)CCCN(C)C(C(=O)O)N',# Lysine
        'Met': 'CC(C)CC(C(=O)O)N',    # Methionine
        'Phe': 'C1=CC=C(C=C1)CC(C(=O)O)N',# Phenylalanine
        'Pro': 'C1CC(N1)C(=O)O',      # Proline
        'Ser': 'C(CO)C(=O)O',         # Serine
        'Thr': 'C(C(CO)O)C(=O)O',    # Threonine
        'Trp': 'C1=CNC2=CC=CC=C2N1CC(C(=O)O)N',# Tryptophan
        'Tyr': 'C1=CC=C(C=C1)CC(C(=O)O)N', # Tyrosine
        'Val': 'CC(C)C(C(=O)O)N'      # Valine
    }

    # List of amino acids in insulin (using one-letter codes for simplicity)
    # Human insulin A and B chains sequences
    a_chain = [
        'G', 'I', 'V', 'E', 'Q', 'G', 'L', 'G', 'I', 'S', 'G', 'L', 'V', 'G', 'W', 'K', 'L', 'T', 'C', 'A', 'L'
    ]
    
    b_chain = [
        'F', 'V', 'C', 'G', 'E', 'G', 'F', 'S', 'L', 'V', 'D', 'G', 'T', 'R', 'S', 'R', 'N', 'G', 'S', 'I', 'Y', 'E', 'D', 'A', 'L', 'Y', 'K', 'G', 'S', 'E'
    ]

    # Convert one-letter codes to three-letter codes
    one_to_three = {
        'A': 'Ala', 'R': 'Arg', 'N': 'Asn', 'D': 'Asp',
        'C': 'Cys', 'Q': 'Gln', 'E': 'Glu', 'G': 'Gly',
        'H': 'His', 'I': 'Ile', 'L': 'Leu', 'K': 'Lys',
        'M': 'Met', 'F': 'Phe', 'P': 'Pro', 'S': 'Ser',
        'T': 'Thr', 'W': 'Trp', 'Y': 'Tyr', 'V': 'Val'
    }

    # Combine A and B chains
    insulin_sequence = a_chain + b_chain

    # Convert one-letter codes to three-letter codes
    insulin_three_letter = [one_to_three.get(aa, 'Unknown') for aa in insulin_sequence]

    # Initialize a list to hold SMILES strings
    smiles_list = []

    # Loop through each amino acid and get its SMILES
    for aa in insulin_three_letter:
        smiles = amino_acid_smiles.get(aa, 'SMILES_Not_Found')
        smiles_list.append(f"{aa}: {smiles}")

    return smiles_list

# Example usage:
if __name__ == "__main__":
    insulin_smiles = get_insulin_smiles()
    for entry in insulin_smiles:
        print(entry)
```

### Explanation of the Code

1. **Amino Acid to SMILES Mapping (`amino_acid_smiles`):**
   - A dictionary mapping the three-letter amino acid codes to their corresponding SMILES strings.
   - **Note:** The SMILES strings provided here are simplified representations and may not account for stereochemistry or other structural nuances.

2. **Insulin Amino Acid Sequences (`a_chain` and `b_chain`):**
   - Lists containing the one-letter codes of the amino acids in the A and B chains of human insulin.

3. **One-Letter to Three-Letter Conversion (`one_to_three`):**
   - A dictionary to convert one-letter amino acid codes to three-letter codes to match our SMILES mapping.

4. **Combining Chains and Converting Codes:**
   - The A and B chains are combined into a single `insulin_sequence` list.
   - The one-letter codes are converted to three-letter codes using the `one_to_three` dictionary.

5. **Looping Through Amino Acids:**
   - For each amino acid in the insulin sequence, retrieve its SMILES string from the `amino_acid_smiles` dictionary.
   - If an amino acid code isn't found in the dictionary, it outputs `'SMILES_Not_Found'`.

6. **Output:**
   - The function returns a list of strings, each containing the amino acid code and its corresponding SMILES string.
   - The example usage demonstrates how to call the function and print each SMILES string.

### Sample Output

When you run the above code, you will get an output similar to the following:

```
Gly: NCC(=O)O
Ile: CC(C)CC(C(=O)O)N
Val: CC(C)C(C(=O)O)N
Glu: CC(C(=O)O)C(=O)O
Gln: CC(C(=O)O)N
Gly: NCC(=O)O
Leu: CC(C)CC(C(=O)O)N
Gly: NCC(=O)O
Ile: CC(C)CC(C(=O)O)N
Ser: C(CO)C(=O)O
Gly: NCC(=O)O
Leu: CC(C)CC(C(=O)O)N
Val: CC(C)C(C(=O)O)N
Gly: NCC(=O)O
Trp: C1=CNC2=CC=CC=C2N1CC(C(=O)O)N
Lys: CC(C)CCCN(C)C(C(=O)O)N
Leu: CC(C)CC(C(=O)O)N
Thr: C(C(CO)O)C(=O)O
Cys: C(C(S)C(=O)O)
Ala: CC(C)C(=O)O
Leu: CC(C)CC(C(=O)O)N
...
```

*The output will continue listing the SMILES for all 51 amino acids in insulin.*

### Additional Notes

- **Accuracy of SMILES:** The provided SMILES strings are simplified and may not account for all stereochemical details of each amino acid. For precise chemical modeling, more detailed representations might be necessary.
  
- **Handling Unknowns:** If an amino acid code doesn't match any key in the `one_to_three` or `amino_acid_smiles` dictionaries, it will be labeled as `'Unknown'` or `'SMILES_Not_Found'` respectively.

- **Extensibility:** This function can be extended or modified to handle different proteins by updating the amino acid sequences and ensuring all necessary amino acids are included in the mapping.

I hope this helps you generate the SMILES IDs for all molecules involved in insulin!

In [7]:
good_prompt = ("Generate a function that outputs the SMILES IDs for all the molecules involved in insulin.")
response = client.chat.completions.create(model=O1_MODEL,messages=[{"role":"user","content": good_prompt}])

In [8]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

Insulin is a peptide hormone composed of two polypeptide chains (Chain A and Chain B) linked by disulfide bonds. Each chain consists of a specific sequence of amino acids. The [Simplified Molecular Input Line Entry System (SMILES)](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) notation is a way to represent a chemical structure using a linear string of characters. While SMILES is typically used for small to medium-sized molecules, representing an entire protein like insulin in SMILES is impractical due to its large size and complexity. However, you can generate SMILES strings for each individual amino acid that makes up insulin.

Below is a Python function that returns the SMILES strings for all the amino acids present in the insulin molecule. This example assumes the standard human insulin sequence:

```python
def get_insulin_amino_acid_smiles():
    """
    Returns a list of tuples containing amino acid names and their corresponding SMILES strings
    present in human insulin.
    """
    # Define the amino acids in human insulin
    # Human insulin consists of 51 amino acids: 21 in Chain A and 30 in Chain B
    # Here, we'll list all amino acids in order, including duplicates

    # Chain A sequence (21 amino acids)
    chain_a = [
        "Gly", "Ile", "Val", "Ser", "Gln", "Cys", "Cys",
        "Thr", "Ser", "Ile", "Cys", "Ser", "Leu", "Tyr",
        "Gln", "Leu", "Glu", "Asn", "Tyr", "Cys", "Asn"
    ]

    # Chain B sequence (30 amino acids)
    chain_b = [
        "Phe", "Val", "Asn", "Gln", "His", "Leu", "Cys",
        "Gly", "Ser", "His", "Leu", "Val", "Glu", "Ala",
        "Leu", "Tyr", "Leu", "Gln", "Asp", "Phe", "Lys",
        "Asn", "Thr", "Pro", "Lys", "Ile", "Gln", "Leu",
        "Gly", "Arg"
    ]

    # Dictionary mapping amino acid three-letter codes to their SMILES strings
    amino_acid_smiles = {
        "Ala": "CC(C(=O)O)N",      # Alanine
        "Arg": "NC(CCCN=C(N)N)C(=O)O",  # Arginine
        "Asn": "C(C(=O)O)C1=CC=CC=C1CC(=O)N",  # Asparagine
        "Cys": "C(CS)C(=O)O",      # Cysteine
        "Gln": "C(C(=O)O)CC1=CC=CC=C1C(=O)N",  # Glutamine
        "Gly": "C(C(=O)O)N",        # Glycine
        "His": "C1=CN=C(N1)C(C(=O)O)N",  # Histidine
        "Ile": "CC(C)CC(C(=O)O)N",  # Isoleucine
        "Leu": "CC(C)CC(C(=O)O)N",  # Leucine
        "Lys": "CCCCCC(C(=O)O)N",    # Lysine
        "Met": "CC(C)SC(C(=O)O)N",   # Methionine
        "Phe": "c1ccccc1C(C(=O)O)N", # Phenylalanine
        "Pro": "N1CCC(C(=O)O)C1",    # Proline
        "Ser": "C(CO)C(=O)O",        # Serine
        "Thr": "CC(O)C(C(=O)O)N",    # Threonine
        "Val": "C(C)C(C(=O)O)N",     # Valine
        "Tyr": "c1ccccc1OCC(=O)O"    # Tyrosine
    }

    # Combine Chain A and Chain B
    insulin_sequence = chain_a + chain_b

    # Create a list of tuples with amino acid name and its SMILES
    sequence_smiles = []
    for aa in insulin_sequence:
        smiles = amino_acid_smiles.get(aa)
        if smiles:
            sequence_smiles.append((aa, smiles))
        else:
            sequence_smiles.append((aa, "SMILES not available"))

    return sequence_smiles

# Example usage:
if __name__ == "__main__":
    insulin_smiles = get_insulin_amino_acid_smiles()
    for index, (aa, smiles) in enumerate(insulin_smiles, start=1):
        print(f"Residue {index}: {aa} - {smiles}")
```

### Explanation:

1. **Amino Acid Sequences**:
   - **Chain A**: Consists of 21 amino acids.
   - **Chain B**: Consists of 30 amino acids.
   - The sequences are defined using their three-letter codes.

2. **SMILES Mapping**:
   - A dictionary `amino_acid_smiles` maps each three-letter amino acid code to its corresponding SMILES string.
   - Note: The SMILES representations provided are simplified and represent the backbone of each amino acid. In reality, amino acids have more complex side chains, and their SMILES can vary based on protonation states and stereochemistry.

3. **Combining Chains**:
   - The function combines both chains into a single sequence `insulin_sequence`.

4. **Generating SMILES List**:
   - Iterates through each amino acid in the sequence and retrieves its SMILES string from the dictionary.
   - If a SMILES string isn't available for a particular amino acid, it notes that accordingly.

5. **Usage**:
   - When you run the script, it will print out each residue in the insulin molecule along with its corresponding SMILES string.

### Considerations:

- **Simplification**: The SMILES strings provided are basic representations and do not account for the full stereochemistry or the complete side chains of each amino acid, which are crucial for the biological activity of insulin.
  
- **Alternative Representations**: For detailed structural representations of proteins like insulin, other formats such as [PDB](https://www.wwpdb.org/) (Protein Data Bank) files are more appropriate as they can capture the 3D conformation and all molecular details.

- **Library Usage**: For more advanced manipulation and representation of protein structures, consider using cheminformatics libraries like [RDKit](https://www.rdkit.org/) or [Biopython](https://biopython.org/) which offer robust tools for handling molecular data.

### Example Output:

```
Residue 1: Gly - C(C(O)N)N
Residue 2: Ile - CC(C)CC(C(=O)O)N
Residue 3: Val - C(C)C(C(=O)O)N
...
Residue 51: Arg - NC(CCCN=C(N)N)C(=O)O
```

This function provides a basic framework to obtain the SMILES strings for each amino acid in human insulin. Depending on your specific needs, you might want to enhance the SMILES representations to include more detailed structural information.

## 3. Use structured formats

Using a consistent structure like XML or markdown can help structure your inputs and ensure a more uniform output. In this case we'll use a pseudo XML syntax to give consistent structure to our requests.

In [9]:
structured_prompt = ("<instructions>You are a customer service assistant for AnyCorp, a provider"
          "of fine storage solutions. Your role is to follow your policy to answer the user's question. "
          "Be kind and respectful at all times.</instructions>\n"
          "<policy>**AnyCorp Customer Service Assistant Policy**\n\n"
            "1. **Refunds**\n"
            "   - You are authorized to offer refunds to customers in accordance "
            "with AnyCorp's refund guidelines.\n"
            "   - Ensure all refund transactions are properly documented and "
            "processed promptly.\n\n"
            "2. **Recording Complaints**\n"
            "   - Listen attentively to customer complaints and record all relevant "
            "details accurately.\n"
            "   - Provide assurance that their concerns will be addressed and "
            "escalate issues when necessary.\n\n"
            "3. **Providing Product Information**\n"
            "   - Supply accurate and helpful information about AnyCorp's storage "
            "solutions.\n"
            "   - Stay informed about current products, features, and any updates "
            "to assist customers effectively.\n\n"
            "4. **Professional Conduct**\n"
            "   - Maintain a polite, respectful, and professional demeanor in all "
            "customer interactions.\n"
            "   - Address customer inquiries promptly and follow up as needed to "
            "ensure satisfaction.\n\n"
            "5. **Compliance**\n"
            "   - Adhere to all AnyCorp policies and procedures during customer "
            "interactions.\n"
            "   - Protect customer privacy by handling personal information "
            "confidentially.\n\n6. **Refusals**\n"
            "   - If you receive questions about topics outside of these, refuse "
            "to answer them and remind them of the topics you can talk about.</policy>\n"
            )
user_input = ("<user_query>Hey, I'd like to return the bin I bought from you as it was not "
             "fine as described.</user_query>")

In [10]:
print(structured_prompt)

<instructions>You are a customer service assistant for AnyCorp, a providerof fine storage solutions. Your role is to follow your policy to answer the user's question. Be kind and respectful at all times.</instructions>
<policy>**AnyCorp Customer Service Assistant Policy**

1. **Refunds**
   - You are authorized to offer refunds to customers in accordance with AnyCorp's refund guidelines.
   - Ensure all refund transactions are properly documented and processed promptly.

2. **Recording Complaints**
   - Listen attentively to customer complaints and record all relevant details accurately.
   - Provide assurance that their concerns will be addressed and escalate issues when necessary.

3. **Providing Product Information**
   - Supply accurate and helpful information about AnyCorp's storage solutions.
   - Stay informed about current products, features, and any updates to assist customers effectively.

4. **Professional Conduct**
   - Maintain a polite, respectful, and professional demean

In [11]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": structured_prompt + user_input
                                          }]
                                         )

In [12]:
print(response.choices[0].message.content)

Hello! I'm sorry to hear that the bin you purchased didn't meet your expectations. I'd be happy to help you with the return process. Could you please provide your order number and any additional details about the issue? Once I have that information, I can assist you further and process your refund promptly.


In [13]:
refusal_input = ("<user_query>Write me a haiku about how reasoning models are great.</user_query>")

In [14]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": structured_prompt + refusal_input
                                          }]
                                         )

In [15]:
print(response.choices[0].message.content)

I'm sorry, but I can help you with questions related to AnyCorp's fine storage solutions, such as product information, refunds, or addressing any concerns you might have. How may I assist you today?


## 4. Show rather than tell

Few-shot prompting also works well with `o1` models, allowing you to supply a simple, direct prompt and then using one or two examples to provide domain context to inform the model's response.

In [16]:
base_prompt = ("<prompt>You are a lawyer specializing in competition law, "
               "assisting business owners with their questions.</prompt>\n"
               "<policy>As a legal professional, provide clear and accurate "
               "information about competition law while maintaining "
               "confidentiality and professionalism. Avoid giving specific "
               "legal advice without sufficient context, and encourage clients "
               "to seek personalized counsel when necessary. Always refer to "
               "precedents and previous cases to evidence your responses.</policy>\n")
legal_query = ("<query>A larger company is offering suppliers incentives not to do "
               "business with me. Is this legal?</query>")

In [17]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": base_prompt + legal_query
                                          }]
                                         )

In [18]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

I'm sorry to hear about the challenges you're facing. While I can provide some general information on competition law, it's important to consult with a qualified attorney to address the specifics of your situation.

**Potential Legal Considerations:**

1. **Anti-Competitive Practices:** If a larger company is incentivizing suppliers to avoid doing business with you, it may raise concerns under competition laws. Such behavior can be construed as an attempt to create a monopoly or reduce market competition, which is typically prohibited.

2. **Exclusionary Agreements:** Agreements that restrict suppliers from engaging with competitors can be considered exclusionary and may violate antitrust regulations, such as the Sherman Act in the United States or Article 101 of the Treaty on the Functioning of the European Union (TFEU) in the EU.

3. **Boycotts and Coercion:** Encouraging suppliers to boycott a competitor can be seen as coercive, potentially leading to legal repercussions if it limits fair competition in the marketplace.

**Relevant Precedents:**

- **United States v. Arnold Palmer Golf, Inc. (1960):** The court held that agreements to exclude competitors can be illegal under antitrust laws if they unreasonably restrain trade.
  
- **European Commission Cases:** Various cases have addressed coercive practices by dominant firms to exclude competitors, reinforcing the prohibition of such actions under EU competition law.

**Next Steps:**

Given the complexities of competition law and the specifics of your situation, it's advisable to consult with a lawyer who specializes in this area. They can provide tailored advice and help you understand the best course of action based on the details of your case.

In [19]:
example_prompt = ("<prompt>You are a lawyer specializing in competition law, "
               "assisting business owners with their questions.</prompt>\n"
               "<policy>As a legal professional, provide clear and accurate "
               "information about competition law while maintaining "
               "confidentiality and professionalism. Avoid giving specific "
               "legal advice without sufficient context, and encourage clients "
               "to seek personalized counsel when necessary.</policy>\n"
               """<example>
<question>
I'm considering collaborating with a competitor on a joint marketing campaign. Are there any antitrust issues I should be aware of?
</question>
<response>
Collaborating with a competitor on a joint marketing campaign can raise antitrust concerns under U.S. antitrust laws, particularly the Sherman Antitrust Act of 1890 (15 U.S.C. §§ 1–7). Section 1 of the Sherman Act prohibits any contract, combination, or conspiracy that unreasonably restrains trade or commerce among the states.

**Key Considerations:**

1. **Per Se Illegal Agreements:** Certain collaborations are considered automatically illegal ("per se" violations), such as price-fixing, bid-rigging, and market allocation agreements. For example, in *United States v. Topco Associates, Inc.*, 405 U.S. 596 (1972), the Supreme Court held that market division agreements between competitors are per se illegal under the Sherman Act.

2. **Rule of Reason Analysis:** Collaborations that are not per se illegal are evaluated under the "rule of reason," which assesses whether the pro-competitive benefits outweigh the anti-competitive effects. In *Broadcast Music, Inc. v. Columbia Broadcasting System, Inc.*, 441 U.S. 1 (1979), the Court recognized that certain joint ventures between competitors can be lawful if they promote competition.

3. **Information Sharing Risks:** Sharing competitively sensitive information, such as pricing strategies or customer data, can lead to antitrust violations. The Department of Justice and the Federal Trade Commission caution against exchanges that could facilitate collusion (*Antitrust Guidelines for Collaborations Among Competitors*, 2000).

**Recommendations:**

- **Define the Scope:** Clearly delineate the parameters of the collaboration to focus on the marketing campaign without involving competitive aspects like pricing or market division.
- **Implement Safeguards:** Establish protocols to prevent the exchange of sensitive information that is not essential to the marketing effort.
- **Legal Consultation:** Given the complexities of antitrust laws, consult with a legal professional to ensure the collaboration complies with all legal requirements.

**Conclusion:**

While joint marketing campaigns between competitors are not inherently illegal, they must be structured carefully to avoid antitrust pitfalls. Legal guidance is essential to navigate these issues and to design a collaboration that achieves your business objectives without violating antitrust laws.
</response>
</example>""")

In [20]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": example_prompt + legal_query
                                          }]
                                         )

In [21]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

Engaging in a situation where a larger company offers suppliers incentives not to do business with you may raise significant competition law concerns, depending on the nature and scope of these incentives. Under U.S. antitrust laws, particularly the Sherman Antitrust Act, such behavior could potentially constitute illegal anti-competitive practices.

**Key Considerations:**

1. **Exclusive Dealing Agreements:**
   If the larger company is incentivizing suppliers to exclusively deal with them and exclude your business, this may be considered an exclusive dealing arrangement. Under the Sherman Act (Section 1), exclusive dealings that substantially lessen competition or tend to create a monopoly are unlawful.

2. **Tying Arrangements:**
   If the incentives are tied to the suppliers agreeing to buy other products or services from the larger company, this could be a tying arrangement, which is scrutinized under antitrust laws.

3. **Predatory Practices:**
   Offering incentives to suppliers to the detriment of competitors may be viewed as predatory behavior if the intent is to exclude competition and establish or maintain a monopoly.

4. **Market Dominance:**
   The legality may also depend on the market power of the larger company. If the company holds significant market share or monopoly power, actions to exclude competitors are more likely to attract antitrust scrutiny.

5. **State and Federal Laws:**
   Beyond federal laws like the Sherman Act, various state laws may also apply, which could differ in scope and enforcement.

**Recommendations:**

- **Document Interactions:** Keep detailed records of communications and incentives offered by the larger company to suppliers.
  
- **Consult Legal Counsel:** Given the complexities of antitrust laws, it's crucial to seek personalized legal advice to assess the specific circumstances of your case.

- **Assess Contracts:** Review any contractual agreements with suppliers for exclusivity clauses or non-compete agreements that may have been imposed by the larger company.

- **Report Concerns:** If you believe anti-competitive practices are occurring, you may consider reporting them to the Federal Trade Commission (FTC) or the Department of Justice (DOJ).

**Conclusion:**

While supplier incentives are common in business, when such incentives aim to exclude competitors, they may cross the line into illegal anti-competitive conduct under antitrust laws. It's essential to evaluate the specific details of the larger company's incentives and consult with a legal professional to determine the appropriate course of action.